In [2]:
# Test the dataset

from lircst_ana_dataset import LircstAnaDataset 

def test_dataset():
    dataset = LircstAnaDataset('/home/samnub/dev/lircst-ana/data/')
    print(len(dataset))
    phan, sino, phantom_id = dataset[0]
    print(phan.shape, sino.shape, phantom_id)

test_dataset()

1797
(2, 128, 128) (128, 200, 100) 1742413505


In [2]:
# Fine-tune ControlNet using pytorch-lightning on our dataset

import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '/home/samnub/dev/lircst-diffusion/src/ControlNet')

from ControlNet.cldm.model import create_model, load_state_dict
from ControlNet.cldm.logger import ImageLogger
from torch.utils.data import DataLoader
from torchvision import transforms
import pytorch_lightning as pl

from ControlNet.tutorial_dataset import MyDataset
from lircst_ana_dataset import LircstAnaDataset

# Configs
resume_path = '/home/samnub/dev/lircst-diffusion/models/control_sd21_ini.ckpt'
batch_size = 4
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = False


# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('/home/samnub/dev/lircst-diffusion/models/cldm_v21.yaml').cpu()
model.load_state_dict(load_state_dict(resume_path, location='cpu'))
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control


# Misc
transform_phan = transforms.Compose([transforms.ToTensor(),
                                     ])
transform_sino = transforms.Compose([transforms.ToTensor(),
                                     ])
# dataset = LircstAnaDataset(data_dir='/home/samnub/dev/lircst-ana/data/')
dataset = MyDataset()
dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)
trainer = pl.Trainer(gpus=1, precision=32, callbacks=[logger])


# Train!
trainer.fit(model, dataloader)

ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 865.91 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


/home/samnub/anaconda3/envs/lircst-diffusion-controlnet/lib/python3.8/site-packages/open_clip/factory.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch

Loaded model config from [/home/samnub/dev/lircst-diffusion/models/cldm_v21.yaml]


/home/samnub/dev/lircst-diffusion/src/ControlNet/cldm/model.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = get_state_dict(torch.load(ckpt_path, map_locati

Loaded state_dict from [/home/samnub/dev/lircst-diffusion/models/control_sd21_ini.ckpt]


FileNotFoundError: [Errno 2] No such file or directory: './training/fill50k/prompt.json'